In [ ]:
from keras.models import load_model
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, ConfusionMatrixDisplay, accuracy_score
import time

import warnings
warnings.filterwarnings("ignore")

Tamanho das imagens

In [ ]:
IMAGE_SIZE = [400, 400]

In [ ]:
train_path = 'ebhi-split-2categorias/train'
valid_path = 'ebhi-split-2categorias/val'
test_path = 'ebhi-split-2categorias/test'

Declarando o modelo VGG16

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3],
            weights='imagenet', include_top=False)

Não treina camadas

In [ ]:
for layer in vgg.layers:
    layer.trainable = False

Adicionando as nossas camadas

In [ ]:
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
#prediction = Dense(len(folders), activation='softmax')(x)
prediction = Dense(1, activation='sigmoid')(x)

Criando o modelo

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)

Visualizando a estrutura do modelo

In [ ]:
model.summary()

Declarando para o modelo as funções de custo e otimização

In [ ]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size=(400, 400),
                                                 batch_size=32,
                                                 class_mode='binary')

In [ ]:
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size=(400, 400),
                                            batch_size=32,
                                            shuffle=False,
                                            class_mode='binary',classes=['ANORMAL','NORMAL'])

In [ ]:
valid_set = valid_datagen.flow_from_directory(valid_path,
                                              target_size=(400, 400),
                                              batch_size=32,
                                              class_mode='binary')

Treinando o modelo

In [ ]:
r = model.fit(
    training_set,
    validation_data=valid_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(valid_set)
)
model.save('hist_model_vgg.h5')

Perda do treino

In [ ]:
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.savefig('LossVal_loss_vgg')
plt.show()

Acurácias do treino

In [ ]:
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.savefig('AccVal_acc_vgg')
plt.show()

Etapa de Testes

In [ ]:
vgg_model = model

t = time.time()
# Usando o modelo para predição das amostras de teste
y_pred = vgg_model.predict(test_set)
# Reset
test_set.reset()
loss, acc = vgg_model.evaluate(test_set)
#aux = np.argmax(aux, axis=1)
y_pred = np.where(y_pred > 0.5, 1, 0).flatten()
print("y predito:")
print(y_pred)
y_true = test_set.classes
print("y real:")
print(y_true)
# Método para calcular o valor F1-Score
print('F1-Score: {}'.format(f1_score(y_true, y_pred, average='macro')))
# Método para calcular a Precision
print('Precision : {}'.format(precision_score(y_true, y_pred, average='macro')))
# Método para calcular o Recall
print('Recall: {}'.format(recall_score(y_true, y_pred, average='macro')))

print('Matriz de Confusão:')
cm = confusion_matrix(y_true, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Anormal','Normal'])
cm_display.plot()
plt.savefig('Matriz-vgg16')
plt.show()

print ('Accuracy score: ', accuracy_score(y_true, y_pred))
print('Acuracia obtida com o Vgg16 no Conjunto de Teste EBHI: {:.2f}'.format(
    acc))



Avaliando no UnitoPatho

In [ ]:

from PIL import Image
from skimage import transform
#funcao carregar imagem
def load(filename):
   np_image = Image.open(filename)
   np_image = np.array(np_image).astype('float32')/255
   np_image = transform.resize(np_image, (400, 400, 3))
   np_image = np.expand_dims(np_image, axis=0)
   return np_image


pred2 = load("dataset-unitopatho/ANORMAL/54-B2-TAHG.ndpi_ROI__mpp0.44_reg000_crop_sk00000_(73992,7343,1812,1812).png")
pred2 = vgg_model.predict(pred2)
print(np.where(pred2 > 0.5, 'Normal', 'Anormal').flatten())


test_path_uni = 'dataset-unitopatho/'
test_datagen_uni = ImageDataGenerator(rescale=1./255)
test_set_uni = test_datagen_uni.flow_from_directory(test_path_uni,
                                            target_size=(400, 400),
                                            batch_size=32,
                                            shuffle=False,
                                            class_mode='binary',classes=['ANORMAL','NORMAL'])



y_pred = vgg_model.predict(test_set_uni)
# Reset
test_set_uni.reset()
loss, acc = vgg_model.evaluate(test_set_uni)
#aux = np.argmax(aux, axis=1)
y_pred = np.where(y_pred > 0.5, 1, 0).flatten()
print("y predito:")
print(y_pred)
y_true = test_set_uni.classes
print("y real:")
print(y_true)
# Método para calcular o valor F1-Score
print('F1-Score: {}'.format(f1_score(y_true, y_pred, average='macro')))
# Método para calcular a Precision
print('Precision : {}'.format(precision_score(y_true, y_pred, average='macro')))
# Método para calcular o Recall
print('Recall: {}'.format(recall_score(y_true, y_pred, average='macro')))


print('Matriz de Confusão:')
cm = confusion_matrix(y_true, y_pred)
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['Anormal','Normal'])
cm_display.plot()
plt.savefig('Matriz-vgg16-UNITOPATHO')
plt.show()

print ('Accuracy score: ', accuracy_score(y_true, y_pred))
print('Acuracia obtida com o Vgg16 no Conjunto de Teste UNITOPATHO: {:.2f}'.format(
    acc))
